In [1]:
import tensorflow as tf
import numpy as np

x_data = [[1, 2, 1, 1],
 [2, 1, 3, 2],
 [3, 1, 3, 4],
 [4, 1, 5, 5],
 [1, 7, 5, 5],
 [1, 2, 5, 6],
 [1, 6, 6, 6],
 [1, 7, 7, 7]]
y_data = [[0, 0, 1],
 [0, 0, 1],
 [0, 0, 1],
 [0, 1, 0],
 [0, 1, 0],
 [0, 1, 0],
 [1, 0, 0],
 [1, 0, 0]]
# 3개의 클래스 분류할때 0,1,2 를 각각 [1,0,0] , [0,1,0], [0,0,1]로 하나만 hot하게 표시함 
x_data = np.asarray(x_data, dtype = np.float32)
y_data = np.asarray(y_data, dtype = np.float32)

#y의 개수 = 클래스 개수 = label개수
dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data)).batch(len(x_data))
W = tf.Variable(tf.random.normal([4,3], name='weight'))
b = tf.Variable(tf.random.normal([3]),name = 'bias')
variable = [W, b]

dataset.element_spec


(TensorSpec(shape=(None, 4), dtype=tf.float32, name=None),
 TensorSpec(shape=(None, 3), dtype=tf.float32, name=None))

In [6]:

def softmax_fn (features):
    hypothesis = tf.nn.softmax(tf.matmul(features,W)+b)
    return hypothesis

def loss_fn (features, labels):
    hypothesis = tf.nn.softmax(tf.matmul(features,W)+b)
    cost = tf.reduce_mean(-tf.reduce_sum(y_data * tf.math.log(hypothesis), axis =1))
    return cost

def grad (hypothesis, features, labels):
    with tf.GradientTape() as tape:
        loss_value = loss_fn(features, labels)
    return tape.gradient(loss_value, [W, b])

optimizer = tf.keras.optimizers.SGD(learning_rate = 0.01)
   
n_epochs = 3000
for step in range(n_epochs + 1):
    
    for features, labels in iter(dataset):
        hypothesis = softmax_fn (features)
        grads = grad(hypothesis, features, labels)
        optimizer.apply_gradients(grads_and_vars = zip(grads, [W, b]))
    
    if step % 300 == 0:
            print("iter: {}, Loss: {:.4f}".format(step, loss_fn(features, labels)))
            
a = x_data
a = softmax_fn (a)
print(hypothesis) #softmax 함수를 통과시킨 x_data

#argmax 가장큰 값의index를 찾아줌

print(tf.argmax(a,1)) #가설을 통한 예측값
print(tf.argmax(y_data,1)) #실제 값

iter: 0, Loss: 0.2747
iter: 300, Loss: 0.2698
iter: 600, Loss: 0.2652
iter: 900, Loss: 0.2607
iter: 1200, Loss: 0.2563
iter: 1500, Loss: 0.2521
iter: 1800, Loss: 0.2480
iter: 2100, Loss: 0.2441
iter: 2400, Loss: 0.2402
iter: 2700, Loss: 0.2365
iter: 3000, Loss: 0.2329
tf.Tensor(
[[8.8713728e-05 4.1324673e-03 9.9577880e-01]
 [4.9200775e-03 1.2180143e-01 8.7327856e-01]
 [2.0201555e-06 2.3868130e-01 7.6131666e-01]
 [9.6342583e-06 7.8007627e-01 2.1991408e-01]
 [3.5828632e-01 6.2282449e-01 1.8889215e-02]
 [1.8675503e-01 8.1322849e-01 1.6514774e-05]
 [6.9866431e-01 3.0125216e-01 8.3541359e-05]
 [8.4900713e-01 1.5098950e-01 3.3782130e-06]], shape=(8, 3), dtype=float32)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)
